In [48]:
import os
import pandas as pd
file_path = './astar_in.txt'
data = pd.read_csv(file_path)

with open(file_path, 'r') as file:
    lines = [l.strip() for l in file.readlines() if len(l.split(' ')) == 3]
    start_list = ' '.join([l for l in lines[:3]]).split(' ')
    start_state = [l.split(' ') for l in lines[:3]]
    end_state = [l.split(' ') for l in lines[3:]]

In [49]:
def display_node(node):
    for i in range(len(node)):
        print(node[i])

### Guide Pseudocode

- Put the start node in the OPEN list
- Check if the current node is already in the CLOSED list
    - If yes, compare the score of the current node vs the node in the closed list
        - If the node in the CLOSED list is lower, use that path
        - If the node in the CLOSED list is higher, exit and choose the next node in the OPEN list
- Check if the start/current node is the goal state
    - If yes, return the solution
    - Otherwise, add the node in the CLOSED list and continue with next step
- Add the current node to the CLOSED list
- Generate the list of expansion states from the start/current node
    - Insert the list in the OPEN list
- Compute the function by adding the ff:
    - The path cost from the start node to each of the expanded states
    - The manhattan distance of each expanded state to the goal state
- The node with the lowest function score becomes the current node

In [ ]:
display_node(start_state)

In [ ]:
display_node(end_state)

In [64]:
import copy

def manhattan_calculator(start_state, end_state):
    manhattan_distance = 0
            
    for list1 in start_state:
        
        for item in list1:
            if item == '*':
                continue
            x1 = list1.index(item)
            y1 = start_state.index(list1)
            
            for list2 in end_state:
                
                if item in list2:
                    x2 = list2.index(item)
                    y2 = end_state.index(list2)
                    manhattan_distance += abs(x2-x1) + abs(y2-y1)
    
    return manhattan_distance

def no_of_wrong_tiles_calculator(node, end_state):
    
    no_of_wrong_tiles = 0
    
    l = 0
    for list in node:
        i = 0
        for item in list:
            if node[l][i] != '*' and node[l][i] != end_state[l][i]:
                no_of_wrong_tiles += 1
            i += 1
        l += 1
    
    return no_of_wrong_tiles

def get_clockwise_expectation(goal_state):
    end_state_1d = []
    clockwise_order = [0, 1, 2, 7, 8, 3, 6, 5, 4]
    for list in end_state:
        for item in list:
            end_state_1d.append(item)
    end_state_1d_clockwise = []
    num = 0
    end_state_1d_clockwise = copy.deepcopy(end_state_1d)
    for i in clockwise_order:
        end_state_1d_clockwise[i] = end_state_1d[num]
        num += 1

    end_state_1d_clockwise1 = copy.deepcopy(end_state_1d_clockwise)
    end_state_1d_clockwise.remove('*')
    end_state_1d_clockwise1.remove('*')
    end_state_1d_clockwise1.append(end_state_1d_clockwise1[0])
    clockwise_expectation = {}
    num = 0
    for i in end_state_1d_clockwise:
        clockwise_expectation[i] = end_state_1d_clockwise1[num+1]
        num += 1
    return clockwise_expectation

def nilsson_sequence_calculator(node, end_state):
    
    clockwise_expectation = get_clockwise_expectation(end_state)
    manhattan_distance = manhattan_calculator(start_state, end_state)
    sequence_score = 0

    for i in range(len(node)):
        for j in range(len(node)):
            if node[i][j] == '*':
                continue
            elif i == 1 and j == 1:
                sequence_score += 1
            else:
                if i == 0 and j < 2 and node[i][j + 1] != clockwise_expectation[node[i][j]]:
                    sequence_score += 2
                elif j == 2 and i < 2 and node[i + 1][j] != clockwise_expectation[node[i][j]]:
                    sequence_score += 2
                elif j > 0 and i == 2 and node[i][j - 1] != clockwise_expectation[node[i][j]]:
                    sequence_score += 2
                elif i > 0 and j == 0 and node[i - 1][j] != clockwise_expectation[node[i][j]]:
                    sequence_score += 2
                    
    nilsson_sequence_score = manhattan_distance + 3 * sequence_score
    return sequence_score

def expand_node(node):
    
    expanded_node_list = []
    
    for list in node:
        if list.count('*'):
            x = list.index('*')
            y = node.index(list)
            break
        else:
            continue
            
    x_left = x - 1
    x_right = x + 1
    y_up = y - 1
    y_down = y + 1
            
    if x_left >= 0:
        
        new_list = copy.deepcopy(node)
        new_list[y][x] = new_list[y][x_left]
        new_list[y][x_left] = '*'
        expanded_node_list.append(new_list)
            
    if x_right <= 2:
        
        new_list = copy.deepcopy(node)
        new_list[y][x] = new_list[y][x_right]
        new_list[y][x_right] = '*'
        expanded_node_list.append(new_list)
        
    if y_up >= 0:
        
        new_list = copy.deepcopy(node)
        new_list[y][x] = new_list[y_up][x]
        new_list[y_up][x] = '*'
        expanded_node_list.append(new_list)
        
    if y_down <= 2:
        
        new_list = copy.deepcopy(node)
        new_list[y][x] = new_list[y_down][x]
        new_list[y_down][x] = '*'
        expanded_node_list.append(new_list)
            
    return expanded_node_list

def goal_checker(node, goal_state):
    if node == goal_state: return True
    else: return False
    
def func_score(node, current_path_cost, goal_state, function_name):
    if function_name == 'manhattan distance':
        return current_path_cost + manhattan_calculator(node, goal_state), manhattan_calculator(node, goal_state)
    if function_name == 'nilsson sequence':
        return current_path_cost + manhattan_calculator(node, goal_state) + 3 * nilsson_sequence_calculator(node, goal_state), nilsson_sequence_calculator(node, goal_state), manhattan_calculator(node, goal_state) + 3 * nilsson_sequence_calculator(node, goal_state)
    if function_name == 'wrong tiles':
        return no_of_wrong_tiles_calculator(node, goal_state), no_of_wrong_tiles_calculator(node, goal_state)

In [ ]:
def main_function(start_state, goal_state, function_name):
    
    current_node = copy.deepcopy(start_state)
    prev_node = copy.deepcopy(start_state)
    OPEN = []
    OPEN_scores = []
    step_costs = []
    current_solution = []
    correct_solution = []
    CLOSED = []
    CLOSED_info = []
    current_path_cost = 0
    OPEN.append(current_node)
    OPEN_scores.append(func_score(current_node, 0, goal_state, function_name)[0])
    step_costs.append(0)
    i = 0
    
    while len(OPEN) > 0:
        i += 1
        CLOSED_dict = {}

        display_node(current_node)
        index_of_current_node = OPEN.index(current_node)
        step_cost_of_current_node = step_costs[index_of_current_node]

        # print("Length of Open:", len(OPEN))
        # print("Length of OPEN_scores:", len(OPEN_scores))
        # print("Length of step_costs:", len(step_costs))
        # print('Open Scores:', OPEN_scores)
        print('g(n) =', step_costs[index_of_current_node], '(Current Step Cost)')
        
        if function_name == 'nilsson sequence':
            print('p(n) =', func_score(current_node, step_cost_of_current_node, goal_state, function_name)[1], '(Manhattan Distance)')
            print('s(n) =', func_score(current_node, step_cost_of_current_node, goal_state, function_name)[1], '(Sequence Score)')
            print('h(n) =', func_score(current_node, step_cost_of_current_node, goal_state, function_name)[2], '(Nilsson Sequence)')
        
        if function_name == 'manhattan distance':
            print('p(n) =', func_score(current_node, step_cost_of_current_node, goal_state, function_name)[1], '(Manhattan Distance)')
            
        if function_name == 'wrong tiles':
            print('m(n) =', func_score(current_node, step_cost_of_current_node, goal_state, function_name)[1], '(Wrong Tiles Count)')
        
        print('f(n) =', OPEN_scores[index_of_current_node], '(Function Score)')
            
        print('search cost:', len(CLOSED))
        print('------------------------------')
        if goal_checker(current_node, goal_state):
            correct_solution = current_solution
            return correct_solution, step_cost_of_current_node
        else:
            CLOSED.append(current_node)
            CLOSED_dict['score'] = func_score(current_node, step_cost_of_current_node, goal_state, function_name)[0]
            CLOSED_dict['sequence'] = copy.deepcopy(current_solution)
            CLOSED_info.append(CLOSED_dict)

        # Removing current node
        OPEN.pop(index_of_current_node)
        OPEN_scores.pop(index_of_current_node)
        step_costs.pop(index_of_current_node)
        current_solution.append(current_node)

        # Expand and add expansions to OPEN only if they
        # have not been met before in CLOSED and they don't
        # improve the previous score.
        expanded_states = expand_node(current_node)
        for expanded_state in expanded_states:
            x = func_score(expanded_state, step_cost_of_current_node + 1, goal_state, function_name)[0]
            if expanded_state in CLOSED or expanded_state in OPEN:
                if expanded_state in CLOSED:
                    y = CLOSED_info[CLOSED.index(expanded_state)].get('score')
                    if y < x:
                        current_solution = CLOSED_info[CLOSED.index(current_node)].get('sequence')
                    else:
                        OPEN.append(expanded_state)
                        OPEN_scores.append(x)
                        step_costs.append(step_cost_of_current_node + 1)
                elif expanded_state in OPEN:
                    y = OPEN_scores[OPEN.index(expanded_state)]
                    if y < x:
                        current_solution = CLOSED_info[CLOSED.index(current_node)].get('sequence')
                    else:
                        OPEN.append(expanded_state)
                        OPEN_scores.append(x)
                        step_costs.append(step_cost_of_current_node + 1)
            else:
                OPEN.append(expanded_state)
                OPEN_scores.append(x)
                step_costs.append(step_cost_of_current_node + 1)

        # Assigning new current node from remaining lowest scoring node in OPEN
        lowest_score_node_index = OPEN_scores.index(min(OPEN_scores))
        current_node = OPEN[lowest_score_node_index]

manhattan_result = main_function(start_state, end_state, 'manhattan distance')

['2', '1', '6']
['4', '*', '8']
['7', '5', '3']
g(n) = 0 (Current Step Cost)
p(n) = 12 (Manhattan Distance)
f(n) = 12 (Function Score)
search cost: 0
------------------------------
['2', '1', '6']
['*', '4', '8']
['7', '5', '3']
g(n) = 1 (Current Step Cost)
p(n) = 11 (Manhattan Distance)
f(n) = 12 (Function Score)
search cost: 1
------------------------------
['2', '1', '6']
['4', '8', '*']
['7', '5', '3']
g(n) = 1 (Current Step Cost)
p(n) = 11 (Manhattan Distance)
f(n) = 12 (Function Score)
search cost: 2
------------------------------
['2', '1', '*']
['4', '8', '6']
['7', '5', '3']
g(n) = 2 (Current Step Cost)
p(n) = 10 (Manhattan Distance)
f(n) = 12 (Function Score)
search cost: 3
------------------------------
['2', '1', '6']
['4', '8', '3']
['7', '5', '*']
g(n) = 2 (Current Step Cost)
p(n) = 10 (Manhattan Distance)
f(n) = 12 (Function Score)
search cost: 4
------------------------------
['2', '1', '6']
['4', '8', '3']
['7', '*', '5']
g(n) = 3 (Current Step Cost)
p(n) = 9 (Manhatta

In [59]:
main_function(start_state, end_state, 'nilsson sequence')

['2', '1', '6']
['4', '*', '8']
['7', '5', '3']
Length of Open: 1
Length of OPEN_scores: 1
Length of step_costs: 1
Open Scores: [60]
f(n) = 60 (Function Score)
g(n) = 0 (Current Step Cost)
p(n) = 16 (Manhattan Distance)
s(n) = 16 (Sequence Score)
h(n) = 60 (Nilsson Sequence)
search cost: 0
------------------------------
['2', '1', '6']
['4', '*', '8']
['7', '5', '3']
Length of Open: 4
Length of OPEN_scores: 4
Length of step_costs: 5
Open Scores: [58, 58, 60, 60]
f(n) = 58 (Function Score)
g(n) = 1 (Current Step Cost)
p(n) = 16 (Manhattan Distance)
s(n) = 16 (Sequence Score)
h(n) = 60 (Nilsson Sequence)
search cost: 2
------------------------------
['2', '1', '6']
['*', '4', '8']
['7', '5', '3']
Length of Open: 5
Length of OPEN_scores: 5
Length of step_costs: 7
Open Scores: [58, 60, 60, 61, 61]
f(n) = 58 (Function Score)
g(n) = 2 (Current Step Cost)
p(n) = 15 (Manhattan Distance)
s(n) = 15 (Sequence Score)
h(n) = 57 (Nilsson Sequence)
search cost: 4
------------------------------
['2', 

In [ ]:
main_function(start_state, end_state, 'wrong tiles')